In [ ]:
from src.services.llm import LLM
from src.services.logical_tree import LogicalTree
from src.utils.build_agent import build_agent, query_based_on_tool_calls, fix_github_error
from typing import Callable, List, Dict, Any
from src.services.agents import Tool_Calls

# from src.services.agents import Agent, Tool
from src.services.custom_tools import custom_tools, clean_training_logs
import glob
import os

os.environ["NO_PROXY"] = "*"

# llm = LLM(model_name="qwen3:8b")

llm = LLM(
    model_name="deepseek-chat",
    llm_url="https://api.deepseek.com/chat/completions",
    api_key="sk-2332c3d16a8d4f4ba1b3503074ba04c5",
    format="openai",
)
# llm = LLM(
#     model_name="gpt-5-2025-08-07",
#     llm_url="https://api.aimlapi.com/v1/chat/completions",
#     api_key="9ce046a9681446c48427b3fe4dd7cdd4",
#     format="openai",
# )
# llm = LLM(
#     model_name="gpt-4o-mini",
#     llm_url="https://api.aimlapi.com/v1/chat/completions",
#     api_key="9ce046a9681446c48427b3fe4dd7cdd4",
#     format="openai",
# )



ct = custom_tools(
    HOST_DIR="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/agent/issue_fix",
    HOST_DATA_DIR="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/data/issue_fix",
    MAIN_DIR="/workspace",
    MAIN_DATA_DIR="/data",
    PYTHON_PATH="python",
    MATLAB_PATH="/Applications/MATLAB_R2023b.app/bin/matlab",
    ENV_NAME="agent1",
    LOCAL_TMP_PATH="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/tmp",
    REMOTE_TMP_PATH="/tmp",
    BASE_ENV="python:3.7",
    PMC_URL="https://pmc.ncbi.nlm.nih.gov/articles/PMC7567795",
    llm=llm,
)
from src.services.llm import load_messages, save_messages
tool_calls_path = "/Users/yuanwen/Desktop/Docker_Environment/intern2/3/code/tool_calls_path.json"
env_tool_calls_path = "/workspace/tool_calls_path.json"
# previous_tool_calls = load_messages(tool_calls_path)
# def update_content_with_clearlog(previous_tool_call, tool_name, tool_id, clearlog_func):
#     for item in previous_tool_call:
#         if item.get("name") == tool_name and item.get("tool_call_id") == tool_id:
#             original_content = item.get("content", "")
#             cleaned_content = clearlog_func(original_content)
#             item["content"] = cleaned_content
#             return cleaned_content
#     return None

# 用法
# cleaned = update_content_with_clearlog(
#     previous_tool_calls,
#     "func_cmd",
#     "call_00_N7b0LUYT8t6svwV9wM7cF2VC",
#     clean_training_logs  # 你之前写好的 clearlog 函数
# )

# print(cleaned)
# # # previous_toll_calls = []
# # # previous_toll_calls.extend(generate_tool_calls(ct.MAIN_DIR))
# save_messages(previous_tool_calls, tool_calls_path)


# def func1(a: int) -> int:
#     return a + 1
# tools = [func1]
# tc = Tool_Calls(PATH="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/test.json")
# llm.query_with_tools(prompt="What is func1(1)+func1(2)?", tc=tc, max_steps=1, tools=tools, verbose=True)
# llm.query("Please tell me who are you")

In [2]:
from src.utils.prompts import get_prompt

package_name = "package"

goal_list = [
    (
        "test github code",
        get_prompt("github_test_prompt.txt").format(
            package_name=package_name, MAIN_DIR=ct.MAIN_DIR
        ),
    ),
    (
        "build agent",
        get_prompt("build_agent_prompt.txt").format(
            package_name=package_name,
            MAIN_DIR=ct.MAIN_DIR,
            MAIN_DATA_DIR=ct.MAIN_DATA_DIR,
        ),
    ),
    (
        "improve agent",
        "try to make the agent better by train a new model by yourself with the training data"
        "and then test it on the test data, compare the result with the original result."
        "You must finally give me a better performance agent (compare to the original one)"
        "You can try anything.",
    ),
]

# goal_list = [("test", "please generate help_return_to_parent directly")]

tc = Tool_Calls(PATH=tool_calls_path, ENV_PATH=env_tool_calls_path, MAX_CHAR=100000)
tc.clear()

message = build_agent(
    goal_list=goal_list,
    ct=ct,
    max_steps=10000,
    tc=tc,
    github_url="https://github.com/lungproject/lungegfr",
    tree_filepath="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/code/logical_tree1.json",
    package_name=package_name,
    verbose=True,
    whether_recreate=False,
    human_decision=True
)


[INFO] Cloning GitHub repository...


[INFO] Starting LLM query for goal 1...

[INFO] Recomputed 0 nodes in total (excluding layer 0).

[LLM] Query with Tools
  [INPUT]
    You are solving a task step by step using a logical tree.
    
    - The **current node** you are working on is:
      - Name: test github code
      - Description: MAIN_DIR/package/ is a GitHub package I downloaded.
    Please run tests with both **real** and **sample** data and output the prediction results.
    You **must compare the prediction results calculated by yourself with the results in the corresponding PMC article by calling func_fetch_info_from_pmc_with_llm**.
    The result should be equal to the paper, but perhaps due to some reason, it may not be correct.
    Any test code you write using tools should be placed under MAIN_DIR/,
    MAIN_DIR=/workspace. You must provide all necessary parameters to the tools.
    When installing dependencies, the main goal is to minimize local resource consumption 
 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
from datasets import load_from_disk
ds = load_from_disk("/Users/yuanwen/Desktop/Docker_Environment/intern2/4/data/SWE-bench_Lite")
print(ds)
sample = ds[0]
package_name = "package"

tc = Tool_Calls(PATH=tool_calls_path, ENV_PATH=env_tool_calls_path, MAX_CHAR=100000)
tc.clear()

message = build_agent(
    sample=sample,
    ct=ct,
    max_steps=10000,
    tc=tc,
    tree_filepath="/Users/yuanwen/Desktop/Docker_Environment/intern2/3/code/logical_tree_fix_issue.json",
    package_name=package_name,
    verbose=True,
    whether_recreate=True,
)

KeyboardInterrupt: 

In [4]:
request = """
你觉得这里有哪些以前调用过的tool calls已经没有用了, 我指的是以前调用过, 比如:
{
    "role": "assistant",
    "content": "",
    "tool_calls": [
      {
        "id": "call_00_aBZLoDEh8lIVr9xgbfiS9COe",
        "type": "function",
        "function": {
          "name": "func_modify",
          "arguments": {
            "filepath": "package/test_allpatient.py",
            "old_text": "np.savetxt(\"./Results/Hebeipredicttrain.txt\",predicttrainpetct1 )",
            "new_text": "np.savetxt(\"/workspace/package/Results/Hebeipredicttrain.txt\",predicttrainpetct1 )"
          }
        },
        "index": 0,
        "name": "func_modify"
      },
      {
        "id": "call_01_ON4j8QJUR3YBW6IeJZzG4GuB",
        "type": "function",
        "function": {
          "name": "func_modify",
          "arguments": {
            "filepath": "package/test_allpatient.py",
            "old_text": "np.savetxt(\"./Results/Hebeipredicttest.txt\",predicttestpetct1 )",
            "new_text": "np.savetxt(\"/workspace/package/Results/Hebeipredicttest.txt\",predicttestpetct1 )"
          }
        },
        "index": 1,
        "name": "func_modify"
      },
      {
        "id": "call_02_BFycuTYRUdoCYBJPu9W6HVUC",
        "type": "function",
        "function": {
          "name": "func_modify",
          "arguments": {
            "filepath": "package/test_allpatient.py",
            "old_text": "np.savetxt(\"./Results/Harbinpdlpredicttest.txt\",predicttestpetct1 )",
            "new_text": "np.savetxt(\"/workspace/package/Results/Harbinpdlpredicttest.txt\",predicttestpetct1 )"
          }
        },
        "index": 2,
        "name": "func_modify"
      }
    ]
  },
  {
    "role": "tool",
    "name": "func_modify",
    "tool_call_id": "call_00_aBZLoDEh8lIVr9xgbfiS9COe",
    "content": "Written to package/test_allpatient.py"
  },
  {
    "role": "tool",
    "name": "func_modify",
    "tool_call_id": "call_01_ON4j8QJUR3YBW6IeJZzG4GuB",
    "content": "Written to package/test_allpatient.py"
  },
  {
    "role": "tool",
    "name": "func_modify",
    "tool_call_id": "call_02_BFycuTYRUdoCYBJPu9W6HVUC",
    "content": "Written to package/test_allpatient.py"
  },
请你直接输出对应tool_call_id(上面的为call_01_ON4j8QJUR3YBW6IeJZzG4GuB)
"""

tc = Tool_Calls(PATH=tool_calls_path)
query_based_on_tool_calls(
    request=request,
    ct=ct,
    tc=tc,
    verbose=True,
)


=== Prompt step 1 ===

Failed to parse JSON from line: -alive
根据之前的tool calls记录，我找到了一个已经没有用的tool call：

**call_01_ON4j8QJUR3YBW6IeJZzG4GuB**

这个tool call是用来修改文件路径的：
- 函数名：func_modify
- 文件路径：package/test_allpatient.py  
- 修改内容：将`"./Results/Hebeipredicttest.txt"`改为`"/workspace/package/Results/Hebeipredicttest.txt"`

这个修改已经执行过并且成功了，现在这个tool call已经没有用了，因为：
1. 文件修改已经完成
2. 后续的测试运行已经使用了新的路径
3. 预测结果已经成功生成在指定位置

'根据之前的tool calls记录，我找到了一个已经没有用的tool call：\n\n**call_01_ON4j8QJUR3YBW6IeJZzG4GuB**\n\n这个tool call是用来修改文件路径的：\n- 函数名：func_modify\n- 文件路径：package/test_allpatient.py  \n- 修改内容：将`"./Results/Hebeipredicttest.txt"`改为`"/workspace/package/Results/Hebeipredicttest.txt"`\n\n这个修改已经执行过并且成功了，现在这个tool call已经没有用了，因为：\n1. 文件修改已经完成\n2. 后续的测试运行已经使用了新的路径\n3. 预测结果已经成功生成在指定位置'

In [6]:
ct.llm.system_prompt = ""
ct.llm.query("测试")


[LLM] Query
  [SYSTEM PROMPT]
    
  [INPUT]
    测试
  [OUTPUT]
    您好！我很乐意为您提供帮助！😊
    
    看起来您是在测试我的功能，这是完全正常的！让我简单介绍一下我能为您做些什么：
    
    **我能帮您：**
    - 回答各种问题和解答疑惑
    - 协助写作、翻译、编程等任务
    - 分析和处理您上传的文件（图片、PDF、Word、Excel等）
    - 进行创意思考和问题解决
    - 提供学习和工作上的支持
    
    **我的特点：**
    - 完全免费使用
    - 支持128K上下文长度
    - 可以联网搜索（需要您手动开启）
    - 知识更新到2024年7月
    
    请随时告诉我您需要什么帮助，或者想测试什么具体功能。我会尽我所能为您提供最好的服务！✨
    
    有什么具体的问题或任务想要尝试吗？


'您好！我很乐意为您提供帮助！😊\n\n看起来您是在测试我的功能，这是完全正常的！让我简单介绍一下我能为您做些什么：\n\n**我能帮您：**\n- 回答各种问题和解答疑惑\n- 协助写作、翻译、编程等任务\n- 分析和处理您上传的文件（图片、PDF、Word、Excel等）\n- 进行创意思考和问题解决\n- 提供学习和工作上的支持\n\n**我的特点：**\n- 完全免费使用\n- 支持128K上下文长度\n- 可以联网搜索（需要您手动开启）\n- 知识更新到2024年7月\n\n请随时告诉我您需要什么帮助，或者想测试什么具体功能。我会尽我所能为您提供最好的服务！✨\n\n有什么具体的问题或任务想要尝试吗？'